In [1]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [2]:
env=gym.make('FrozenLake-v1', render_mode="rgb_array")

In [3]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
print(action_size)
print(state_size)

4
16


In [6]:
total_episodes = 10000        # Total episodes
learning_rate = 0.1           # Learning rate
max_steps = 100                # Max steps per episode
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001            # Exponential decay rate for exploration prob

# **Train Q-learning Agent**

In [7]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()[0]
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done,truncated, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print("\n\n********Q-table********\n")
print(qtable)

Score over time: 0.5228


********Q-table********

[[0.51329166 0.51474613 0.51255712 0.50981977]
 [0.25975836 0.24155629 0.2640451  0.50092822]
 [0.41017541 0.39179561 0.40640843 0.46356181]
 [0.31349698 0.32470601 0.35603312 0.44401677]
 [0.5234935  0.43703955 0.40150909 0.40152917]
 [0.         0.         0.         0.        ]
 [0.19987321 0.17785401 0.23994246 0.13030262]
 [0.         0.         0.         0.        ]
 [0.22983864 0.40584947 0.42579766 0.56135255]
 [0.47943235 0.61666816 0.40537442 0.43043192]
 [0.56005878 0.39597524 0.33370591 0.35967781]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.42630026 0.56524552 0.73396556 0.66544908]
 [0.71891243 0.89163693 0.71364791 0.741018  ]
 [0.         0.         0.         0.        ]]


In [8]:
rewards_per_thousand_episodes = np.split(np.array(rewards),total_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.046000000000000034
2000 :  0.20400000000000015
3000 :  0.3890000000000003
4000 :  0.5590000000000004
5000 :  0.6490000000000005
6000 :  0.6820000000000005
7000 :  0.6940000000000005
8000 :  0.6730000000000005
9000 :  0.6530000000000005
10000 :  0.6790000000000005


In [9]:

for episode in range(3):
    state = env.reset()[0]
    done = False
    print("****************************************************")
    print("EPISODE ", episode+1)
    time.sleep(1)
    for step in range(max_steps):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break
        state=new_state
env.close()

****You reached the goal!****
